In [ ]:
%matplotlib inline

import healpy as hp
import glob
from scipy.optimize import curve_fit
import pickle
from importlib import reload
import time
import scipy


# Specific qubic modules
from qubicpack.utilities import Qubic_DataDir
from pysimulators import FitsArray
import pysm
import qubic
from qubic import QubicSkySim as qss
from qubic import fibtools as ft
from qubic import camb_interface as qc
from qubic import SpectroImLib as si
from qubic import NamasterLib as nam
from qubic import mcmc

reload(qss)
reload(ft)

rc('figure', figsize=(16, 10))
rc('font', size=15)
mpl.rcParams['image.cmap'] = 'jet'

This notebook aims at doing the same study as in FastSimulation-CMB bt turning ON the spectro-imaging. This means that on top of using a modeling of the RMS Vs. coverage and noise spatial correlation, we need to include sub-bands covariance matrices that are to be provided by Louise Mousset.

As a first attempt we will make the following assumptions (to be checked on End-To-End data):
- the coverage shape remains the same
- the evolution of noise w.r.t. coverage remains the same
- the spatial correlations remain the same
- we do not have I,Q,U correlations.

So we will just include some sub-bands intercoorelations in this forst work.

In [ ]:
### Initialize
global_dir = Qubic_DataDir(datafile='instrument.py', datadir=os.environ['QUBIC_DATADIR'])

dictfilename = global_dir + '/dicts/BmodesNoDustNoSystPaper0_2020.dict'


# Read dictionary chosen
d = qubic.qubicdict.qubicDict()
d.read_from_file(dictfilename)
d['nside']=256
center = qubic.equ2gal(d['RA_center'], d['DEC_center'])
print(center)

In [ ]:
#### This function reads the maps
def read_files_mc(dirmap,file_rnd_string, verbose=False):
    m = np.array(FitsArray(glob.glob(dirmap+'*_maps_recon_*'+file_rnd_string+'.fits')[0]))
    c = np.array(FitsArray(glob.glob(dirmap+'*_maps_convolved_*'+file_rnd_string+'.fits')[0]))
    cov = np.array(FitsArray(glob.glob(dirmap+'*_maps_coverage_*'+file_rnd_string+'.fits')[0]))
    with open(glob.glob(dirmap+'*_dictionary_'+file_rnd_string+'.pickle')[0], 'rb') as handle: d = pickle.load(handle)
    with open(glob.glob(dirmap+'*_input_cell_'+file_rnd_string+'.pickle')[0], 'rb') as handle: icl = pickle.load(handle)
    filetype = glob.glob(dirmap+'*_maps_recon_*'+file_rnd_string+'.fits')[0].split('/')[-1]
    if verbose: print('read {}'.format(filetype))
    return m, c, cov, d, icl, filetype

### Get reconstructed maps from a simulation made on NERSC with 200000 pointings and tol=1e-5
### First Instrument
### detector_nep=4.7e-17
### effecgtive_duration = 3 years
nptg = 200000
dirmaps='/Users/hamilton/Qubic/QubicGeneralPaper2020/Sims/SimsPureCMB_NERSC/Maps/'
file_rnd_string='H990hwzaGn'
tol=1e-5

m, c, cov, d, icl, ftype = read_files_mc(dirmaps, file_rnd_string, verbose=True)

nbins = 100   
reload(qss)
xx, myY, effective_variance_invcov = qss.get_noise_invcov_profile((m[0,:,:]-c[0,:,:]), 
                                                                  cov, 0.1, nbins=nbins, doplot=True,
                                                                  fit=True, norm=True, allstokes=True,
                                                                 label='QUBIC MC Average IQU Nptg={}  Tol={}'.format(nptg,tol))
plot(effective_variance_invcov[0,:], np.sqrt(effective_variance_invcov[1,:]),'--')
legend()

## Make a Fast Realization including Spatial Correlations

Note: one should not be worried by the fact that the RMS is so different in the Fast SImualtions, this is once again related to missing modes that are discussed in "2pt_Correlation Function.Rmd". They do have the right power spectrum in the range we are interested in, it's just that becasue they are simulated in ell sapce, there is some aliasing effect that is different for noise simualted in map space and ell space...

In [ ]:
### Now do a realization of QUBIC sky
reload(qss)
reload(qc)
## Make a sky using PYSM: It will have the expected QUBIC beam, the coverage and noise according to this coverage
seed = None
sky_config = {'cmb': seed}
Qubic_sky = qss.Qubic_sky(sky_config, d)

## Read Spatial Noise Correlation file
clnoise = pickle.load( open( global_dir+'scripts/QubicGeneralPaper2020/cl_corr_noise_nersc200k.pk', "rb" ) )
alpha = 4.5 ### See notebook called "2pt-Correlation Function" for an empirical explanation of alpha
clnoise = (clnoise -1 ) * alpha + 1

subplot(1,2,1)
plot(clnoise)

signoise = 75
qubicnoise = Qubic_sky.create_noise_maps(signoise, cov, 
                                         effective_variance_invcov=effective_variance_invcov,clnoise=None)

subplot(1,2,2)
xx, yyfs, bla = qss.get_noise_invcov_profile(qubicnoise, cov, label='FastSim', fit=False, allstokes=True, doplot=True)
xx, yysim, bla = qss.get_noise_invcov_profile(m[0,:,:]-c[0,:,:], cov, 
                                   label='QUBIC MC Average IQU Nptg={}  Tol={}'.format(nptg,tol), fit=False, doplot=True)
plot(effective_variance_invcov[0,:], np.sqrt(effective_variance_invcov[1,:])*yysim[0],'--',label='Qubic Law')
xlim(0,12)
ylim(0,2.5)
legend(loc='upper left', fontsize=10)

Now we need to assume a shape for the sub-freqeuncies covariance matrices...

For now we assume an optimistic scaling of sqrt(nsub) for the overall RMS (which is currently not verified in the simulations, where there apparently is something more like nsub scaling (irk !). Maybe the negative correlation with neaby bad that is obbserved mitigates this...

Below is the first model we try along with useful functions:

In [ ]:
# this is the scaling for the C00 element in nu,nu covariance matrix

# this si the correlation matrix for nsub sub-bands
def corr_nunu(nsub, nearest=-0.5):
    mycorr = np.diag(np.ones(nsub))
    for i in range(nsub-1):
        mycorr[i,i+1] = nearest
        mycorr[i+1,i] = nearest
    return mycorr

def cov_nunu(nsubm, diagvals, nearest=-0.5):
    test_corr = corr_nunu(nsub, nearest=nearest)
    return qss.corr2cov(test_corr, diagvals)


nsub = 4
test_corr = corr_nunu(nsub)
diagvals = 1+(np.random.rand(nsub)-0.5)*0.5
#diagvals = np.ones(nsub)
subs = np.arange(nsub)

rc('figure', figsize=(16, 6))
subplot(1,2,1)
plot(subs, diagvals,'ro')
xlabel('Sub-Frequency')
ylabel(r'$C_{00}$')

subplot(1,2,2)
imshow(test_corr)
xlabel('Sub-Frequency')
ylabel('Sub-Frequency')
title('Correlation Matrix')
plt.xticks(subs)
plt.yticks(subs)
colorbar()

figure()
test_cov = cov_nunu(nsub, diagvals)
back_corr = qss.cov2corr(test_cov)

subplot(1,3,1)
imshow(test_cov)
xlabel('Sub-Frequency')
ylabel('Sub-Frequency')
title('Covariance Matrix (normalized)')
plt.xticks(subs)
plt.yticks(subs)
colorbar()

subplot(1,3,2)
imshow(back_corr)
title('Back to Cor Matrix')
xlabel('Sub-Frequency')
ylabel('Sub-Frequency')
plt.xticks(subs)
plt.yticks(subs)
colorbar()
subplot(1,3,3)
imshow(back_corr-test_corr)
title('DIfference')
xlabel('Sub-Frequency')
ylabel('Sub-Frequency')
plt.xticks(subs)
plt.yticks(subs)
colorbar()

tight_layout()

## Now we need to simulate according to this covariance matrix

In [ ]:
diagvals = 1+(np.random.rand(nsub)-0.5)*0.5
test_cov = cov_nunu(nsub, diagvals)

### Eigenvectors of the correlation matrix
mymatrix = test_cov
w, v = numpy.linalg.eig(mymatrix)
print('initial matrix:')
print(mymatrix)
print()
print('Eigenvalues')
print(w)
print()
print('Eigenvectors')
print(v)
print()
print('Diagonalized with eigenvectors')
diagmatrix = np.dot(np.dot(v.T,mymatrix),v)
print(diagmatrix)
print()
print('Now back again to inital space')
backmatrix = np.dot(np.dot(v,diagmatrix),v.T)
print(backmatrix)

subplot(2,3,1)
imshow(mymatrix)
title('Initial Matrix')

subplot(2,3,2)
imshow(diagmatrix)
title('Diagonalized Matrix')

subplot(2,3,3)
imshow(backmatrix)
title('Back Matrix')

subplot(2,1,2)
plot(np.ravel(mymatrix), np.ravel(backmatrix),'ro')
plot(np.linspace(np.min(mymatrix),np.max(mymatrix),10), np.linspace(np.min(mymatrix),np.max(mymatrix),10),'k:')
xlabel('Initial Matrix')
ylabel('Back Matrix')

So in principle the method is as follows:
1. One simulates unocorrelated maps in the space where the correlation matrix is diagonal, withe the scaling given by its eigenvalues.
2. These maps are then rotated in the initial space
3. they should have the right covariance matrix...

Let's first try with a map with normal distributed pixels in the coverage

In [ ]:
def plotall(cov_out, cov_th, corr_out, corr_th, tit=''):
    rc('figure', figsize=(16, 6))

    subplot(2,4,1)
    imshow(cov_out, vmin=np.min(cov_th), vmax = np.max(cov_th))
    title('Output Cov '+tit)
    colorbar()

    subplot(2,4,2)
    imshow(cov_th, vmin=np.min(cov_th), vmax = np.max(cov_th))
    title('Th Cov '+tit)
    colorbar()

    subplot(2,4,3)
    imshow(cov_out-cov_th,vmin=np.min(corr_th), vmax = np.max(corr_th))
    title('Diff Cov '+tit)
    colorbar()

    subplot(2,4,4)
    plot(np.ravel(cov_out), np.ravel(cov_th), 'ro')
    plot(np.linspace(-0.5,1,100), np.linspace(-0.5,1,100),'k:')
    xlabel('Th Cov '+tit)
    ylabel('Output Cov '+tit)

    subplot(2,4,5)
    imshow(corr_out, vmin=np.min(corr_th), vmax = np.max(corr_th))
    title('Output Corr '+tit)
    colorbar()


    subplot(2,4,6)
    imshow(corr_th, vmin=np.min(corr_th), vmax = np.max(corr_th))
    title('Th Corr '+tit)
    colorbar()

    subplot(2,4,7)
    imshow(corr_out-corr_th,vmin=np.min(corr_th), vmax = np.max(corr_th))
    title('Diff Corr '+tit)
    colorbar()

    subplot(2,4,8)
    plot(np.ravel(corr_out), np.ravel(corr_th), 'ro')
    plot(np.linspace(-0.5,1,100), np.linspace(-0.5,1,100),'k:')
    xlabel('Th Corr '+tit)
    ylabel('Output Corr '+tit)

    tight_layout()    

In [ ]:
### With a partial map but with flat coverage
sub_maps = np.zeros((nsub, 12*d['nside']**2, 3))
new_maps = np.zeros((nsub, 12*d['nside']**2, 3))
okpix = cov > 0

all_cov_in = []
all_cov_out = []
for s in range(3):
    for i in range(nsub):
        sub_maps[i,okpix,s] = np.random.randn(okpix.sum()) * np.sqrt(w[i])

    all_cov_in.append(np.cov(sub_maps[:,okpix,s]))
    new_maps[:, okpix, s] = np.dot(v, sub_maps[:,okpix, s])
    all_cov_out.append(np.cov(new_maps[:, okpix,s]))

plotall(all_cov_out[0], mymatrix, qss.cov2corr(all_cov_out[0]), qss.cov2corr(mymatrix), tit='I')
figure()
plotall(all_cov_out[1], mymatrix, qss.cov2corr(all_cov_out[1]), qss.cov2corr(mymatrix), tit='Q')
figure()
plotall(all_cov_out[2], mymatrix, qss.cov2corr(all_cov_out[2]), qss.cov2corr(mymatrix), tit='U')


OK this is fine ! Now next step: into map simulator...

In [ ]:
### Now do a realization of QUBIC sky
reload(qss)
reload(qc)
## Make a sky using PYSM: It will have the expected QUBIC beam, the coverage and noise according to this coverage
seed = None
sky_config = {'cmb': seed}
Qubic_sky = qss.Qubic_sky(sky_config, d)

signoise_global = 75
nsub = 4
signoise_each = signoise_global/np.sqrt(nsub)
allpix = cov > 0

nsub = 5

##### None case
# my_cov = None

##### Indentity Case
# my_cov = np.diag(np.ones(nsub))

##### General case
diagvals = np.ones(nsub)#+(np.random.rand(nsub)-0.5)*0.5
test_cov = cov_nunu(nsub, diagvals)
my_cov = test_cov / test_cov[0,0]

sub_maps = Qubic_sky.create_noise_maps(signoise_each, cov, nsub=nsub,
                                                 effective_variance_invcov=effective_variance_invcov,
                                                 clnoise=clnoise, sub_bands_cov=my_cov, verbose=True)



print('Variance Sub maps')
print(np.var(sub_maps,axis=1))

print()
print('Target Cov I:')
print(my_cov)
print()
print('Target Cov Q,U:')
if my_cov is not None: print(my_cov*2)

np.set_printoptions(precision=2)
reload(qss)
cov_I, cov_Q, cov_U, allfitcov, allnormnoise = qss.get_cov_nunu(sub_maps, cov)

print()
print('Cov_I')
print(cov_I)
print()
print('Cov_Q')
print(cov_Q)
print()
print('Cov_U')
print(cov_U)


allcov = [cov_I, cov_Q, cov_U]

if nsub > 1:
    rc('figure', figsize=(16, 8))
    for s in range(3):
        subplot(3,3,s+1)
        imshow(allcov[s], vmin=-0.1, vmax=2)
        colorbar()
        title('Cov '+sn[s])
        if my_cov is not None:
            subplot(3,3,s+4)
            if s==0:
                p=1
            else:
                p=2.
            imshow(my_cov*p, vmin=-0.1, vmax=2)
            colorbar()
            title('Cov Th '+sn[s])
            subplot(3,3,s+7)
            imshow(allcov[s]-my_cov*p, vmin=-0.1, vmax=2)
            colorbar()
            title('Diff '+sn[s])

    tight_layout()

Let's try to simulate a map from its cov and fitcov function

In [ ]:
reload(qss)
reload(qc)
## Make a sky using PYSM: It will have the expected QUBIC beam, the coverage and noise according to this coverage
seed = None
sky_config = {'cmb': seed}
Qubic_sky = qss.Qubic_sky(sky_config, d)

#### First simulate as usual
nsub = 4
signoise_global = 75.
signoise_each = signoise_global*np.sqrt(nsub)
diagvals = np.ones(nsub)
test_cov = cov_nunu(nsub, diagvals)
my_cov = test_cov / test_cov[0,0]

print(effective_variance_invcov[1,:10])

sub_maps = Qubic_sky.create_noise_maps(signoise_each, cov, nsub=nsub,
                                                 effective_variance_invcov=effective_variance_invcov,
                                                 clnoise=None, sub_bands_cov=my_cov, verbose=True)



for i in range(nsub):
    xx, yyfs, bla = qss.get_noise_invcov_profile(sub_maps[i,:,:], cov, 
                                                 label='Input Map {}'.format(i+1), fit=False,
                                                 allstokes=False, doplot=True)
plot(effective_variance_invcov[0,:], np.sqrt(effective_variance_invcov[1,:])*yyfs[0],'r--', label='Input Profile Cov')
legend(fontsize=9)
xlim(0.9,10.1)
title('Un-Normalized')

In [ ]:
reload(qss)
reload(qc)


### We take these maps as an input and try to reproduce them again (as one would do with real data)
# calculate covariance
cI, cQ, cU, fitcov, noise_norm = qss.get_cov_nunu(sub_maps, cov)

figure()
subplot(1,2,1)
for i in range(nsub):
    xx, yyfs, bla = qss.get_noise_invcov_profile(sub_maps[i,:,:], cov, 
                                                 label='Input Map {}'.format(i+1), fit=False, norm=True,
                                                 allstokes=False, doplot=True)
    plot(fitcov[i][0,:], np.sqrt(fitcov[i][1,:]/fitcov[i][1,0]), label='fit on {}'.format(i))
plot(effective_variance_invcov[0,:], np.sqrt(effective_variance_invcov[1,:]),'r--', label='Input Profile Cov')
xlim(0.9,10.1)
title('Input maps profile - Normalized')
legend()

subplot(1,2,2)
for i in range(nsub):
    xx, yyfs, bla = qss.get_noise_invcov_profile(sub_maps[i,:,:], cov, 
                                                 label='Input Map {}'.format(i+1), fit=False, norm=False,
                                                 allstokes=False, doplot=True)
    plot(fitcov[i][0,:], np.sqrt(fitcov[i][1,:]), label='fit on {}'.format(i))
#plot(effective_variance_invcov[0,:], np.sqrt(effective_variance_invcov[1,:]),'r--', label='Input Profile Cov')
xlim(0.9,10.1)
title('Input maps profile - Un-Normalized')
legend()



figure()
subplot(1,4,1)
imshow(my_cov, vmin=-1, vmax=1)
colorbar(orientation='horizontal')
title('Input')
subplot(1,4,2)
imshow(cI, vmin=-1, vmax=1)
colorbar(orientation='horizontal')
title('Map I')
subplot(1,4,3)
imshow(cQ/2, vmin=-1, vmax=1)
colorbar(orientation='horizontal')
title('Map Q / 2')
subplot(1,4,4)
imshow(cU/2, vmin=-1, vmax=1)
colorbar(orientation='horizontal')
title('Map U / 2')

# ### Now reproduce maps from this
# ### Note: the second should not be scaled with signoise_each as we should deduce its normalization 
# ### from the other variables, siilarly it should not depend on nsub for the same reason
# ### However there remains a normalization here that is not clear...
# ### There is always a factor 84 lower for the new wrt to inital (if new has signoise=1)

fitcov_norm = []
for i in range(nsub):
    bla = fitcov[i].copy()
    bla[1,:] = fitcov[i][1,:] / fitcov[i][1,0]
    fitcov_norm.append(bla)


#### This works OK but we should not know signoise_each...
seed = None
sky_config = {'cmb': seed}
Qubic_sky = qss.Qubic_sky(sky_config, d)
print('Normalization sigma_sec = {}'.format(signoise_each))
new_sub_maps = Qubic_sky.create_noise_maps(signoise_each, cov, nsub=nsub,
                                            effective_variance_invcov=fitcov_norm,
                                            clnoise=None, sub_bands_cov=my_cov, verbose=True)





figure()
subplot(1,2,1)
for i in range(nsub):
    xx, yyfs, bla = qss.get_noise_invcov_profile(new_sub_maps[i,:,:], cov, 
                                                 label='Input Map {}'.format(i+1), fit=False, norm=True,
                                                 allstokes=False, doplot=True)
    plot(fitcov[i][0,:], np.sqrt(fitcov[i][1,:]/fitcov[i][1,0]), label='fit on {}'.format(i))
plot(effective_variance_invcov[0,:], np.sqrt(effective_variance_invcov[1,:]),'r--', label='Input Profile Cov')
xlim(0.9,10.1)
title('New maps profile - Normalized')
legend()

subplot(1,2,2)
for i in range(nsub):
    xx, yyfs, bla = qss.get_noise_invcov_profile(new_sub_maps[i,:,:], cov, 
                                                 label='Input Map {}'.format(i+1), fit=False, norm=False,
                                                 allstokes=False, doplot=True)
    plot(fitcov[i][0,:], np.sqrt(fitcov[i][1,:]), label='fit on {}'.format(i))
#plot(effective_variance_invcov[0,:], np.sqrt(effective_variance_invcov[1,:]),'r--', label='Input Profile Cov')
xlim(0.9,10.1)
title('New maps profile - Un-Normalized')
legend()




# print()
# print(np.std(sub_maps[:,okpix,0]))
# print(np.std(new_sub_maps[:,okpix,0]))
# print(np.std(sub_maps[:,okpix,0])/np.std(new_sub_maps[:,okpix,0]))

newcI, newcQ, newcU, newfitcov, norm_noise = qss.get_cov_nunu(new_sub_maps, cov)
figure()
subplot(1,4,1)
imshow(my_cov, vmin=-1, vmax=1)
title('Input')
colorbar(orientation='horizontal')
subplot(1,4,2)
imshow(newcI, vmin=-1, vmax=1)
title('New Map I')
colorbar(orientation='horizontal')
subplot(1,4,3)
imshow(newcQ/2, vmin=-1, vmax=1)
title('New Map Q / 2')
colorbar(orientation='horizontal')
subplot(1,4,4)
imshow(newcU/2, vmin=-1, vmax=1)
title('New  Map U / 2')
colorbar(orientation='horizontal')

figure()
for i in range(nsub):
    subplot(2,nsub, i+1)
    xx, yyfs, bla = qss.get_noise_invcov_profile(sub_maps[i,:,:], cov, label='Innput Map {}'.format(i+1), fit=False, allstokes=False, doplot=True)
    xx, yyfs, bla = qss.get_noise_invcov_profile(new_sub_maps[i,:,:], cov, label='Output Map {}'.format(i+1), fit=False, allstokes=False, doplot=True)
    legend(fontsize=9)
    plot(fitcov[i][0,:], np.sqrt(fitcov[i][1,:]), label='fit on {}'.format(i))
tight_layout()



OK this works fine ! SO we can go for a script to run at Lyon

# Script for Lyon

In [ ]:
#### Testing the script
########################################################################################################
#### Now in a function to loop over ell binning, lmin, coverage
def run_mc_spectroim(nsub, nbmc, Namaster, d, signoise, cov, effective_variance_invcov, verbose=False, clnoise=None):
    ell_bins, b = Namaster.get_binning(d['nside'])
    mask_apo = Namaster.mask_apo
    okpix = cov > (np.max(cov)*0.1)

    myd = d.copy()
    myd['nf_sub']=nsub
    seed = np.random.randint(1,100000)
    sky_config = {'cmb': seed}
    Qubic_sky = qss.Qubic_sky(sky_config, myd)

    w=None
    cl_noise_qubic = np.zeros((nbmc, nsub, len(ell_bins), 4))
    print('        Starting MC')
    for imc in range(nbmc):
        t0 = time.time()
        qubicnoiseA = Qubic_sky.create_noise_maps(signoise, cov, nsub=nsub,
                                                  effective_variance_invcov=effective_variance_invcov,
                                                 clnoise=clnoise)
        qubicnoiseB = Qubic_sky.create_noise_maps(signoise, cov, nsub=nsub,
                                                  effective_variance_invcov=effective_variance_invcov,
                                                 clnoise=clnoise)
        

        for isub in range(nsub):
            ### Compute Spectra:
            # Noise Only
            print('   - QUBIC Noise maps sub {}'.format(isub))
            leff, cl_noise_qubic[imc, isub, :,:], w = Namaster.get_spectra(qubicnoiseA[isub,:,:].T, 
                                                                     map2 = qubicnoiseB[isub,:,:].T,
                                                                     purify_e=False, purify_b=True, w=w, verbose=False,
                                                                     beam_correction=True)
        t1 = time.time()
        print('             Monte-Carlo: Iteration {0:} over {1:} done in {2:5.2f} sec'.format(imc, nbmc,t1-t0))
        
    
    # average MC results
    return leff, cl_noise_qubic


In [ ]:
dirdata = '/Users/hamilton/Qubic/QubicGeneralPaper2020/'
cov_file = dirdata + 'Coverage_Nersc200k.fits'
file_noise_profile = dirdata + 'fit_noise_profile_nersc200k.pk'
clnoise_file = dirdata + 'cl_corr_noise_nersc200k.pk'
outname = '/Users/hamilton/Qubic/QubicGeneralPaper2020/Sims/SpectroIm/FastSim/'

is_spatial_corr = 0
covcut = 0.1
lmin = 40
delta_ell = 30
nbmc = 3
nsub = 4

In [ ]:
### Initialize
global_dir = Qubic_DataDir(datafile='instrument.py', datadir=os.environ['QUBIC_DATADIR'])
dictfilename = global_dir + '/dicts/BmodesNoDustNoSystPaper0_2020.dict'
# Read dictionary chosen
d = qubic.qubicdict.qubicDict()
d.read_from_file(dictfilename)
d['nside'] = 256
center = qubic.equ2gal(d['RA_center'], d['DEC_center'])

### Open Coverage File
cov = np.array(FitsArray(cov_file))

### Open Noise Profile
fit_n200k = pickle.load( open( file_noise_profile, "rb" ) )

### Open Cl for spatially  correlated noise
if is_spatial_corr == 0:
    print('No Spatial Correlations')
    clth = None
else:
    print('Spatial correlations from '+clnoise_file)
    clth = pickle.load( open( clnoise_file, "rb" ) )
    alpha = 4.5 ### See notebook called "2pt-Correlation Function" for an empirical explanation of alpha
    clth = (clth -1 ) * alpha + 1

print('Spatial correlations:')
print(clth)



################################### Flat Weighting #################################################################
### Create a Namaster object
lmax = 2 * d['nside'] - 1
okpix = cov > np.max(cov) * covcut

### We use Flat weighting
maskpix = np.zeros(12*d['nside']**2)
maskpix[okpix] = 1
Namaster = nam.Namaster(maskpix, lmin=lmin, lmax=lmax, delta_ell=delta_ell)

### Run the MC
leff, cl_noise_qubic = run_mc_spectroim(nsub, nbmc, Namaster, d, signoise, cov, 
                                        fit_n200k, clnoise=clth)
### Save Output
rndstr = qss.random_string(10)
outfile = outname + 'MCFastNoiseSpectroIm_nsub_{}_n_{}_sig_{}_lmin_{}_dl_{}_cc_{}_str_{}_Flat.pk'.format(nsub, nbmc, signoise, lmin, delta_ell, covcut, rndstr)
#pickle.dump([leff, cl_noise_qubic, sys.argv], open(outfile, "wb"))
pickle.dump([leff, cl_noise_qubic], open(outfile, "wb"))
####################################################################################################################
